In [1]:
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from os.path import join
import pandas as pd
from utilities import basic, utils, mri_stats, stats
import numpy as np

In [2]:
table_dir = join(base_dir, 'data/tables')
df = utils.load_scan_csv(join(table_dir, 'neg_pos.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  


### Select Manufacturer and B0 

In [31]:
df = df[df.Manufacturer=='GE MEDICAL SYSTEMS']
df = df.dropna(subset=['MagneticFieldStrength'])

4070

### Get patients with DWI AND T2*GRE

In [36]:
mask_dict, tag_dict = mri_stats.get_masks_dict(df)

#### Lets use only the Series Description, not the predicted image) 

In [93]:
dwi_tags = ['DWI', 'DTI']
df_dwi = df[stats.check_tags(df, dwi_tags)]
df_t2s_gre = df[mask_dict['t2s'] & mask_dict['gre']]
dwi_pat = set(df_dwi.PatientID.unique())
t2s_gre_pat = set(df_t2s_gre.PatientID.unique())
pat_t2s_dwi = np.array(list(dwi_pat.intersection(t2s_gre_pat)))

#### Save the patient list 

In [95]:
np.savetxt("D:\\Thesis\\Cobra\\cobra\\data\\share\\Cerebriu\\download_patients\\ge_dwi_t2s_gre.txt",
          pat_t2s_dwi,delimiter=" ", fmt="%s")

In [75]:
df_comb = df[df.PatientID.isin(pat_t2s_dwi)]

In [96]:
df_comb[df_comb.PatientID==pat_t2s_dwi[4]].SeriesDescription

152733                                      Sag 3D FIESTA-C
152734                                               Ax DWI
152735                                           Ax T2* GRE
152736    ADC (10^-6 mmÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃ...
152737                                SAG T2 512 FSE (Flow)
152738                                      Ax T2 PROPELLER
152739                                      Cor T2 FLAIR FS
152740                                         Sag T1 FLAIR
Name: SeriesDescription, dtype: object

In [80]:
df_comb.MagneticFieldStrength.value_counts()

1.5    375
3.0     26
Name: MagneticFieldStrength, dtype: int64